# L5c: Support Vector Machines (SVM)
In this lecture, we will explore Support Vector Machines (SVM) and their kernelized versions. The SVM is a supervised learning algorithm used for classification and regression tasks. 

> __Learning Objectives:__
> 
> By the end of this lecture, you should be able to:
> Three learning objectives go here. 

Let's get started!
___

## Theory: Support Vector Machine (SVM)
Suppose, we have dataset $\mathcal{D} = \{(\hat{\mathbf{x}}_{i}, y_{i}) \mid i = 1,2,\dots,n\}$, where $\hat{\mathbf{x}}_i \in \mathbb{R}^p$ is an _augmented_ feature vector ($m$ features with additional `1` to model the bias on the end of the vector) and $y_i \in \{-1, 1\}$ is the corresponding class label.

> __What is the goal of an SVM?__ 
> 
> The goal of an SVM is to find the hyperplane $\mathcal{H}(\hat{\mathbf{x}}) = \{\hat{\mathbf{x}} \mid \left<\hat{\mathbf{x}},\theta\right> = 0\}$ that separates the data points into two classes (those points above the hyperplane, and those points below the hyperplane), where $\theta \in \mathbb{R}^{p}$ ($p=m+1$) is the normal vector to the hyperplane, or alternatively, the parameters of the model that we need to estimate.


Support vector machines (SVMs) and other approaches, e.g., [the perceptron](https://en.wikipedia.org/wiki/Perceptron) differ primarily in their optimization objectives and training methods: while a [perceptron](https://en.wikipedia.org/wiki/Perceptron) can find _a hyperplane_ that separates classes, SVMs seek to find the _best hyperplane_ in the sense that the _margin_ between classes is maximized.

There are (at least) two strategies that we could use to estimate the unknown parameters $\theta \in \mathbb{R}^{p}$, depending upon if we know beforehand whether the dataset $\mathcal{D}$ is linearly separable.

Let's start with the case where we know that the dataset is linearly separable.


<div>
    <center>
        <img src="figs/Fig-SVM-Schematic.svg" width="480"/>
    </center>
</div>

### Hard margin case: Linearly separable data
If the data is linearly separable, a hyperplane $\mathcal{H}(\hat{\mathbf{x}})$ exists that perfectly seperates the data. We can estimate the _best_ hyperplane by maximizing the _margin_, which is equivalent to minimizing the parameter vector $\theta$. 

> __Hard Margin Problem__
> 
> The __maximum hard margin problem__ for a support vector classifier is given by:
> $$
\boxed{
\begin{align*}
    \min_{\theta}\quad & \frac{1}{2}\lVert{\theta}\rVert_{2}^{2}\\
    \text{subject to}\quad & y_{i}\left<\hat{\mathbf{x}}_{i},\theta\right> \geq 1\quad\forall i
\end{align*}}
> $$
> where $\theta\in\mathbb{R}^{p}$ denote the unknown parameters that we are trying to estimate, $\hat{\mathbf{x}}_{i}\in\mathbb{R}^{p}$ are the augmented (training) feature vectors, $y_{i}\in\{-1,1\}$ are the class labels, and $p=m+1$ is the number of parameters, where $m$ is the number of features. The index $i$ runs over the training examples, i.e., one constraint per training example.

__Do we solve the hard margin problem?__

__Yes!__ We can solve the hard margin problem above using [quadratic programming](https://en.wikipedia.org/wiki/Quadratic_programming) (QP) techniques. In fact, many off-the-shelf optimization packages (e.g., [CVXOPT](https://cvxopt.org/), [OSQP](https://osqp.org/), [Gurobi](https://www.gurobi.com/), etc.) can solve QP problems efficiently.

However, in practice, we rarely know beforehand whether the data is linearly separable. Therefore, we typically use a more general formulation of the SVM that can handle non-linearly separable data. Let's explore that next.
___

<div>
    <center>
        <img src="figs/Fig-SVM-Schematic-Softmargin.svg" width="480"/>
    </center>
</div>

## Soft margin case: Not linearly separable 
If the data is _not linearly separable_, then we know that a perfect $\mathcal{H}(\hat{\mathbf{x}})$ will not exist, i.e., no hyperplane will separate the data without making at least one mistake. In this case, we can estimate the _best_ hyperplane possible by solving the maximum soft margin problem given by:

> __Soft Margin Problem__
>
> The _maximum soft margin problem_ for a support vector classifier is given by:
> $$
\begin{align*}
    \min_{\theta}\quad & \frac{1}{2}\lVert{\theta}\rVert_{2}^{2} + C\sum_{i=1}^{n}\xi_{i}\\
    \text{subject to}\quad & y_{i}\left<\hat{\mathbf{x}}_{i},\theta\right> \geq 1 - \xi_{i}\quad\forall i\\
    & \xi_{i} \geq 0\quad\forall i
\end{align*}
> $$
> where $\xi_{i}$ is a _slack variable_, that quantifies the cost of a classification mistake, and $C>{0}$ is a user-adjustable parameter that controls the trade-off between maximizing the margin and minimizing the slack variables.

__Values of the hyperparameter C__: If $C\gg{1}$ the classifier will behave more like the maximum (hard) margin classifier, i.e., mistakes will be expensive, and the search will avoid making choices with mistakes. However, if $C\ll{1}$, the classifier will allow more slack (mistakes), i.e., mistakes are cheap, so what's it matter!

__Do we solve the soft margin problem?__

__No!__ Typically, we don't solve the problem above directly; instead, we reformulate it as an _unconstrained_ problem [using a hinge-loss function](https://en.wikipedia.org/wiki/Hinge_loss):
$$
\begin{equation*}
    \min_{\theta}\left[\frac{1}{2}\lVert{\theta}\rVert_{2}^{2} + C\sum_{i=1}^{n}\max\{0, 1 - y_{i}\left<\hat{\mathbf{x}}_{i},\theta\right>\}\right]
\end{equation*}
$$
where the sum is computed over $n$ training examples. Yet again application of a penalty term to the objective function! Here, the penalty term is the _hinge loss function_, which penalizes misclassifications and points that are within the margin.

__How do we solve this?__ We can solve this unconstrained optimization problem using a variety of techniques, including [gradient descent](https://en.wikipedia.org/wiki/Gradient_descent), [stochastic gradient descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent), or hueristic methods like [genetic algorithms](https://en.wikipedia.org/wiki/Genetic_algorithm) or [simulated annealing](https://en.wikipedia.org/wiki/Simulated_annealing). Many off-the-shelf optimization packages can also solve this problem efficiently.
___

## Kernelized SVM
In the previous sections, we assumed that the data could be separated by a linear hyperplane. However, in many real-world scenarios, the data may not be linearly separable in its original feature space. To address this, we can use the _kernel trick_ to implicitly map the data into a higher-dimensional space where it may be linearly separable.

To apply the kernel trick to the soft margin classifier, we return to the (non-augmented) feature vector $\mathbf{x}_{i}\in\mathbb{R}^{m}$ and keep the bias term $b$ explicit. Let $\phi(\mathbf{x})$ denote the feature map associated with the kernel $K(\mathbf{x}_{i},\mathbf{x}_{j}) = \left<\phi(\mathbf{x}_{i}),\phi(\mathbf{x}_{j})\right>$. The soft margin problem in feature space becomes:

> __Kernelized Soft Margin Problem (primal)__
>
> The _kernelized soft margin problem_ can be written as:
> $$
\begin{align*}
    \min_{\theta, b, \xi}\quad & \frac{1}{2}\lVert{\theta}\rVert_{2}^{2} + C\sum_{i=1}^{n}\xi_{i}\\
    \text{subject to}\quad & y_{i}\left(\left<\phi(\mathbf{x}_{i}),\theta\right> + b\right) \geq 1 - \xi_{i}\quad\forall i\\
    & \xi_{i} \geq 0\quad\forall i
\end{align*}
> $$
> where $\theta$ is the normal vector in feature space, $b$ is the bias, and $\xi_i$ are the slack variables.

__Do we solve the primal?__ Almost never. In practice, we rarely have an explicit feature map $\phi(\cdot)$ (or it is too high-dimensional to construct), so solving the primal would defeat the purpose of the kernel trick. If an explicit, low-dimensional map is available and cheap to evaluate, then the primal is an option; otherwise we move to the dual.

Solving the primal requires explicit access to $\phi(\cdot)$, which we want to avoid. Instead, we switch to the dual, where the inner products become kernel evaluations:

> __Kernelized Soft Margin Problem (dual)__
>
> The dual (and the one we actually solve) is:
> $$
\begin{align*}
    \max_{\alpha}\quad & \sum_{i=1}^{n}\alpha_{i} - \frac{1}{2}\sum_{i=1}^{n}\sum_{j=1}^{n}\alpha_{i}\alpha_{j}y_{i}y_{j}K(\mathbf{x}_{i},\mathbf{x}_{j})\\
    \text{subject to}\quad & 0 \leq \alpha_{i} \leq C\quad\forall i\\
    & \sum_{i=1}^{n}\alpha_{i}y_{i} = 0
\end{align*}
> $$
> where $\alpha_i$ are the Lagrange multipliers (one per training example).

__Decision function.__ After solving for $\alpha$, we classify new points using:
$$
\begin{equation*}
    f(\mathbf{x}) = \sum_{i=1}^{n}\alpha_{i}y_{i}K(\mathbf{x}_{i},\mathbf{x}) + b,\qquad \hat{y} = \text{sign}\{f(\mathbf{x})\}
\end{equation*}
$$
Only points with $\alpha_{i} > 0$ appear in the sum; these are the _support vectors_.

> __KKT reminder.__ Complementary slackness gives $\alpha_i\left[y_i f(\mathbf{x}_i) - 1 + \xi_i\right]=0$ and $0\le \alpha_i \le C$. Therefore, if a point satisfies $y_i f(\mathbf{x}_i) > 1$ (strictly outside the margin) then $\alpha_i=0$, while points on or inside the margin have $\alpha_i>0$.

For any support vector with $0 < \alpha_{k} < C$, we can compute the bias term using:
$$
\begin{equation*}
    b = y_{k} - \sum_{i=1}^{n}\alpha_{i}y_{i}K(\mathbf{x}_{i},\mathbf{x}_{k})
\end{equation*}
$$

__Do we solve this?__ Yes! The dual is a quadratic program that depends only on the kernel matrix $K(\mathbf{x}_{i},\mathbf{x}_{j})$, so we can use the same QP machinery as before without ever computing $\phi(\cdot)$ explicitly.
___

## Summary
One direct, concice summary sentence goes here.

> __Key Takeaways:__
> 
> THree key takeaways go here.

One direct, concise concluding sentence goes here.
___